# GPT (Generative Pretrained Transformer)

In [ ]:
#| default_exp bigram 

In [ ]:
#|export
import requests
from fastcore.all import L, urlsave
from rich import print
import numpy
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(1337)

<torch._C.Generator>

In [ ]:
#|export
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Read & Explore Data

In [ ]:
#|export
path = urlsave('https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt', '../data/input.txt')

In [ ]:
#|export
text = path.read_text(encoding='utf-8')

In [ ]:
print("Length of the dataset ", len(text))

Length of the dataset  1115394

In [ ]:
# lets look at the first 1000 characters 
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.

## Tokenization

In [ ]:
#|export
chars = L(set(text)).sorted()
vocab_size = len(chars)

In [ ]:
print(''.join(chars), vocab_size)

!$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz 65

In [ ]:
#|export
stoi = {ch: i for i, ch in enumerate(chars)}
itos = dict(map(reversed, stoi.items()))
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

In [ ]:
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]

hii there

In [ ]:
#|export
# lets now encode the entire text dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
print(data.shape, data.dtype)
print(data[:100]) # the 100 characters we looked at earlier will look like this to the GPT

torch.Size([1115394])
torch.int64

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

## Train/Validation Split

In [ ]:
#|export
# lets now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest will be val
train_data = data[:n]
val_data = data[n:]

## Dataloader: Batches of chunks of data

In [ ]:
block_size = 8 # also known as context length
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47

when input is tensor([18, 47]) the target: 56

when input is tensor([18, 47, 56]) the target: 57

when input is tensor([18, 47, 56, 57]) the target: 58

when input is tensor([18, 47, 56, 57, 58]) the target: 1

when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15

when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47

when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58

In [ ]:
#|export
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split, batch_size = batch_size, block_size = block_size):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y        

In [ ]:
xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)
print('--------')
    
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs

torch.Size([4, 8])

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

targets

torch.Size([4, 8])

...

when input is [25, 17, 27, 10] the target: 0

when input is [25, 17, 27, 10, 0] the target: 21

when input is [25, 17, 27, 10, 0, 21] the target: 1

when input is [25, 17, 27, 10, 0, 21, 1] the target: 54

when input is [25, 17, 27, 10, 0, 21, 1, 54] the target: 39

In [ ]:
print(xb) # our input to the transformer

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])

## Simplest Baseline: Bigram Language Model, Loss, Generation

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads of the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        return logits

In [ ]:
m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
out.shape

torch.Size([4, 8, 65])

#### Introduce cross entropy loss

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads of the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        # Cross entropy expects C in second dimension 
        B, T, C = logits.shape
        logits = logits.view(B * T, C)
        targets = targets.view(B * T)
        loss = F.cross_entropy(logits, targets) 
        
        return logits, loss

In [ ]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
logits.shape, loss

(torch.Size([32, 65]), tensor(4.7288, grad_fn=<NllLossBackward0>))

#### Introduce generation function

In [ ]:
#|export
class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B, T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        if targets is None:
            loss = None
        else:
            # Cross entropy expects C in second dimension
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
            
        return idx

In [ ]:
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
logits.shape, loss

(torch.Size([32, 65]), tensor(4.2793, grad_fn=<NllLossBackward0>))

In [ ]:
idx = torch.zeros((1, 1), dtype = torch.long)
out = m.generate(idx, max_new_tokens=100); 
out.shape, out

(torch.Size([1, 101]),
 tensor([[ 0, 25, 59, 44, 30,  9,  2, 63, 15,  1, 26, 20, 23, 19, 37, 12, 15,  9,
          43, 16, 17, 19, 25, 54,  7, 34, 15, 41, 38, 26,  9, 39, 53, 30, 26,  4,
          54, 53, 61, 44, 36, 51, 14,  8, 17, 27, 36,  8, 39, 18, 50,  9, 57, 47,
          58, 19, 48,  4, 35, 41, 19, 54, 53, 17, 44, 30, 60, 20, 24, 40, 30, 12,
           0, 22, 62, 20, 53, 44, 30, 36,  6, 25, 41, 41, 13, 35, 14, 32, 18,  8,
          13, 24, 18,  3, 51, 29, 11,  2, 19, 37, 10]]))

In [ ]:
decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist())

"\n\n$mxmncZSB 'yQOcZqIsKso-y'LhFdr\nFDMwr,GY!KitFz:lLvdDM?VPNMH&XvU?QAw;!C.F&fR\n,!YL  LCi;SKVxIpU$ZkI$fV"

### Train the model

#### Create a PyTorch Optimizer

In [ ]:
m = BigramLanguageModel(vocab_size)

In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
bs = 32
for steps in range(15_000):
    
    # sample a batch of data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.6547441482543945

In [ ]:
decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=300)[0].tolist())

'\n\n\nMoutug;\n\nYor gre t we IAme s fasonsu totheinoadouthe arpllle ME antt.\n\nA:\nMOforsouthofursse,\nPeraf. tt tungorecis gengenalasuno led my nstail?\n\nLqu,\nBYONENulinf ste brth tielothomairuthely bbes baty bune wim s itaigge ph thee chado ser: tatldorot d\n\nCA:\nnd ck\n\nDurek s,\n\nHoff toeenosther; w.\n\n\nT:\nL'

#### Train the model

In [ ]:
#|export

# hyperparameters
max_iters = 3000 
eval_interval = 300
lr = 1e-2
eval_iters = 200

In [ ]:
#|export 

@torch.no_grad() # we will not be calling .backward() in this function 
#so no storage of gradient compuration required
def estimate_loss():
    out = {}
    model.eval() # https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # setting back to training phase
    return out    

In [ ]:
#|export

model = BigramLanguageModel(vocab_size)
m = model.to(device)

# create a PyTorch optimizer    
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    
for step in range(max_iters):

    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.7540, val loss 4.7581

step 300: train loss 3.2280, val loss 3.2440

step 600: train loss 2.7701, val loss 2.7757

step 900: train loss 2.6287, val loss 2.6373

step 1200: train loss 2.5333, val loss 2.5632

step 1500: train loss 2.5162, val loss 2.5909

step 1800: train loss 2.5410, val loss 2.5257

step 2100: train loss 2.5005, val loss 2.5164

step 2400: train loss 2.4846, val loss 2.5060

step 2700: train loss 2.4913, val loss 2.5216

In [ ]:
decode(m.generate(torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist())

"\nur I bck YCLe chiatweentoffigardy,\nItunng, mat by of arsbed te\nTrer n d ndoug tencoumosofomul acharourieat ooy t the kncer f3BETargqsinginthe, f; m alp,\nT:\nBusMerraiks;  nchis lousheer S: ver ar pe I butimagixEN;3EXms n ain peefstheventit, pr; w ue tthantsentowarthat bre intureawit mer:\nCOOLLUCHeJ;\n\n\n\n\nSe.\nG; ces\nWhounthtode mikerelt prty le.\n\nGBUCE:\nTh K:\nA: IUSe lll ls an t it cr t Hod in d.\n\nYBUKI:\nHeaberusinde, t my llfo me boss wss?\nPAn hois ak-mee uswhe alsousil alpano IOBle't IOLYCI's I w"

## The mathematical trick in self-attention

### Version 1
> Averaging past context with for loops

In [ ]:
# consider the following toy example:

B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b, t] = mean_{i <= t} x[b, i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b, t] = torch.mean(xprev, 0)

In [ ]:
x[0]

tensor([[-0.3724, -0.6920],
        [ 0.9520,  0.1971],
        [-1.3562,  0.0523],
        [-0.3947, -0.1389],
        [-1.0761, -0.2606],
        [ 0.2385, -0.6055],
        [-0.4021, -1.3019],
        [-0.2927, -0.7348]])

In [ ]:
xbow[0]

tensor([[-0.3724, -0.6920],
        [ 0.2898, -0.2474],
        [-0.2589, -0.1475],
        [-0.2928, -0.1454],
        [-0.4495, -0.1684],
        [-0.3348, -0.2413],
        [-0.3444, -0.3928],
        [-0.3380, -0.4355]])

In [ ]:
a = torch.ones(3, 3)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

--

b=

tensor([[2., 3.],
        [5., 1.],
        [2., 4.]])

--

c=

tensor([[9., 8.],
        [9., 8.],
        [9., 8.]])

--

In [ ]:
c.shape

torch.Size([3, 2])

### Version 2
> Using Matrix Multiply

In [ ]:
torch.tril(torch.ones(3, 3)) # return lower triangular portion of `torch.ones`

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
a = torch.tril(torch.ones(3, 3))
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b 
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

--

b=

tensor([[9., 9.],
        [4., 8.],
        [6., 6.]])

--

c=

tensor([[ 9.,  9.],
        [13., 17.],
        [19., 23.]])

--

In [ ]:
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3, 2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)
print('--')

a=

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

--

b=

tensor([[5., 6.],
        [9., 4.],
        [4., 8.]])

--

c=

tensor([[5.0000, 6.0000],
        [7.0000, 5.0000],
        [6.0000, 6.0000]])

--

In [ ]:
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

### Version 3
> Adding softmax

In [ ]:
tril = torch.tril(torch.ones(T, T))

In [ ]:
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
print('Before applying softmax')
print(wei)
wei = F.softmax(wei, dim=-1)
print('After applying softmax')
print(wei)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

Before applying softmax

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

After applying softmax

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

True

In [ ]:
xbow3.shape

torch.Size([4, 8, 2])

### Version 4: Self-attention

In [ ]:
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x) # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) * head_size ** -0.5 # (B, T, 16) @ (B, 16, T) ----> (B, T, T)


tril = torch.tril(torch.ones(T, T))

wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v= value(x)
out = wei @ v 

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4864, 0.5136, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3831, 0.4545, 0.1624, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2285, 0.2868, 0.2623, 0.2224, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1658, 0.2953, 0.1889, 0.2168, 0.1333, 0.0000, 0.0000, 0.0000],
         [0.1888, 0.1896, 0.1532, 0.1688, 0.1315, 0.1681, 0.0000, 0.0000],
         [0.1749, 0.1420, 0.1786, 0.1344, 0.1194, 0.1528, 0.0979, 0.0000],
         [0.1478, 0.2097, 0.1018, 0.1039, 0.0677, 0.1179, 0.1829, 0.0682]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5270, 0.4730, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3362, 0.3400, 0.3238, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3764, 0.1737, 0.2026, 0.2473, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1912, 0.1518, 0.1428, 0.2452, 0.2691, 0.0000, 0.0000, 0.0000],
         [0.1751, 0.182

## Code Cleanup

In [ ]:
batch_size = 32
n_embd = 32
lr = 1e-3
max_iters = 5000 

In [ ]:
class Head(nn.Module):
    """one head of self-attention"""
    
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
    
    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # (B, T, C)
        q = self.query(x) # (B, T, C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2, -1) * C**-0.5 # (B, T, C) @ (B, C, T) --> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        # perform the weighted aggregation of the values
        v = self.value(x)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out
        

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = Head(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # (B, T, C)
        x = self.sa_head(x) # apply one head of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the prediction
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
            
        return idx

In [ ]:
model = BigramLanguageModel()

In [ ]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (position_embedding_table): Embedding(8, 32)
  (sa_head): Head(
    (key): Linear(in_features=32, out_features=32, bias=False)
    (query): Linear(in_features=32, out_features=32, bias=False)
    (value): Linear(in_features=32, out_features=32, bias=False)
  )
  (lm_head): Linear(in_features=32, out_features=65, bias=True)
)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for step in range(max_iters):

    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2372, val loss 4.2362

step 300: train loss 3.1707, val loss 3.2214

step 600: train loss 2.9718, val loss 2.9668

step 900: train loss 2.8036, val loss 2.8417

step 1200: train loss 2.7233, val loss 2.7152

...

step 3600: train loss 2.5053, val loss 2.5298

step 3900: train loss 2.4838, val loss 2.5162

step 4200: train loss 2.4904, val loss 2.4838

step 4500: train loss 2.4573, val loss 2.4814

step 4800: train loss 2.4428, val loss 2.4762

## Multi-Headed Self-Attention

In [ ]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    
    def forward(self, x):
        return torch.cat([h(x) for h in self.heads], dim=-1)

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # (B, T, C)
        x = self.sa_head(x) # apply one head of self-attention (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the prediction
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
            
        return idx

In [ ]:
model = BigramLanguageModel()

In [ ]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(65, 32)
  (position_embedding_table): Embedding(8, 32)
  (sa_head): MultiHeadAttention(
    (heads): ModuleList(
      (0): Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
      (1): Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
      (2): Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=False)
        (value): Linear(in_features=32, out_features=8, bias=False)
      )
      (3): Head(
        (key): Linear(in_features=32, out_features=8, bias=False)
        (query): Linear(in_features=32, out_features=8, bias=Fals

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for step in range(max_iters):

    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2068, val loss 4.2074

step 300: train loss 3.1004, val loss 3.1549

step 600: train loss 2.8834, val loss 2.9003

step 900: train loss 2.7915, val loss 2.7893

step 1200: train loss 2.6677, val loss 2.6816

...

step 3600: train loss 2.4768, val loss 2.4678

step 3900: train loss 2.4641, val loss 2.4798

step 4200: train loss 2.4594, val loss 2.4723

step 4500: train loss 2.4383, val loss 2.4152

step 4800: train loss 2.3968, val loss 2.4232

In [ ]:
class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, n_embd),
            nn.ReLU(),
        )
    
    def forward(self, x):
        return self.net(x)

In [ ]:
class BigramLanguageModel(nn.Module):
    
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.sa_head = MultiHeadAttention(4, n_embd//4) # i.e. 4 heads of 8-dimensional self-attention
        self.ffwd = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        # idx and targets are both (B, T) tensor of integers
        tok_embd = self.token_embedding_table(idx) # (B, T, C)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        x = tok_embd + pos_embd # (B, T, C)
        x = self.sa_head(x) # apply one head of self-attention (B, T, C)
        x = self.ffwd(x)
        logits = self.lm_head(x) # (B, T, vocab_size)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the prediction
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim = -1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T + 1)
            
        return idx

In [ ]:
model = BigramLanguageModel()

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

for step in range(max_iters):

    if step % eval_interval == 0:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.2088, val loss 4.2084

step 300: train loss 3.1161, val loss 3.0921

step 600: train loss 2.8387, val loss 2.8419

step 900: train loss 2.7163, val loss 2.7022

step 1200: train loss 2.6280, val loss 2.6622

...

step 3600: train loss 2.4272, val loss 2.4579

step 3900: train loss 2.4080, val loss 2.4583

step 4200: train loss 2.4366, val loss 2.4158

step 4500: train loss 2.3975, val loss 2.3997

step 4800: train loss 2.3576, val loss 2.3640

In [ ]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
    
    def forward(self, x):
        x = x + self.sa(x)
        x = x + self.ffwd(x)
        return x

In [ ]:
class BigramLanguageModel(nn.Module):
        
        def __init__(self):
            super().__init__()
            # each token directly reads off the logits for the next token from a lookup table
            self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
            self.position_embedding_table = nn.Embedding(block_size, n_embd)
            self.blocks = nn.Sequential(
                Block(n_embd, n_head=4),
                Block(n_embd, n_head=4),
                Block(n_embd, n_head=4)
            )
            self.lm_head = nn.Linear(n_embd, vocab_size)
            
        
            
        